In [1]:
import time
import os
import pandas as pd
import numpy as np


import requests
from bs4 import BeautifulSoup



import mysql
import mysql.connector as MC



In [2]:
def store_and_rename_csv(path):
    
    data = pd.read_csv(path)
#I rename thoses columns to have a correct format for the databse 
    data.rename(columns={'team kpm': 'team_kpm', 'earned gpm': 'earned_gpm', 'dragons (type unknown)': 'dragons_ukt'}, inplace=True)
    return data
    
#All function about the creation of tables    
    
def create_game_info(data):
    
    game_info_table = data[["gameid", "playerid", "teamid", "datacompleteness", "url", "league", "year", "split", "playoffs", "date", "patch", "gamelength", "result"]]
    game_info_table.loc[game_info_table['patch'].isnull(), 'patch'] = 0    
    return game_info_table

def create_player_info(data):
    player_info_table = data[[ "playerid", "gameid", "playername", "position", "teamid"]]
    return player_info_table

def create_draft(data):
    draft_table = data[[ "gameid","champion","participantid","playerid","ban1","ban2","ban3","ban4","ban5","side"]]
    return draft_table

def create_team_info(data):
    team_info = data[["teamid","playerid","side","teamname","league"]]
    return team_info

#The function which calls every others function and summuraize on a tuple

def sum_all_table(data):
    info_game_table = create_game_info(data)

    info_player_table = create_player_info(data)

    draft_table = create_draft(data)

    team_info_table = create_team_info(data)
    
    return info_game_table, info_player_table, draft_table, team_info_table

#Here we export the dataframe into the specific file where is my sql script in the mysql file

def export_to_csv(data, name_table = "1"):
    path = 'C:\MyWamp\mysql\script\league_of_legend'
    output_file = os.path.join(path, name_table + '.csv')
    data.to_csv(output_file, index = False)
    return 0
    
    
#This function execute SQl file which create the database, the tables and import all data
    
def executeScriptsFromFile(filename):
    with open(filename, 'r') as sql_file:
        result_iterator = cursor.execute(sql_file.read(), multi=True)
        for res in result_iterator:
            print("Running query: ", res)  # Will print out a short representation of the query
            print(f"Affected {res.rowcount} rows" )


In [3]:
G2022 = store_and_rename_csv("https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2022_LoL_esports_match_data_from_OraclesElixir_"+time.strftime('%Y%m%d')+".csv")

In [4]:
info_game_table, info_player_table, draft_table, team_info_table = sum_all_table(G2022)

In [5]:
export_to_csv(info_player_table, "info_player_table")
export_to_csv(info_game_table, "info_game_table")
export_to_csv(draft_table, "draft_table")
export_to_csv(team_info_table, "team_info_table")

0

In [8]:
#Here I connect to my Database and i lauch the script to actualise the data in my database
try:
    conn = mysql.connector.connect(host= "localhost", database = "lol", user="root", password="")
    cursor = conn.cursor()
    
    executeScriptsFromFile('C:/MyWamp/mysql/script/league_of_legend/lol_games.sql')
    executeScriptsFromFile('C:/MyWamp/mysql/script/league_of_legend/import_data.sql')
    
    
except  MC.Error as err:
    print(err)
finally:
    if (conn.is_connected()):
        cursor.close()
        conn.close()


Running query:  CMySQLCursor: USE lol
Affected 0 rows
Running query:  CMySQLCursor: DROP TABLE IF EXISTS game_info
Affected 0 rows
Running query:  CMySQLCursor: DROP TABLE IF EXISTS player_info
Affected 0 rows
Running query:  CMySQLCursor: DROP TABLE IF EXISTS draft
Affected 0 rows
Running query:  CMySQLCursor: DROP TABLE IF EXISTS team_info
Affected 0 rows
Running query:  CMySQLCursor: CREATE TABLE `game_info` (
  `gameid` VA..
Affected 0 rows
Running query:  CMySQLCursor: CREATE TABLE `player_info` (
  `playerid..
Affected 0 rows
Running query:  CMySQLCursor: CREATE TABLE `draft` (
  `gameid` VARCHA..
Affected 0 rows
Running query:  CMySQLCursor: CREATE TABLE `team_info` (
  `teamid` VA..
Affected 0 rows
Running query:  CMySQLCursor: USE lol
Affected 0 rows
Running query:  CMySQLCursor: LOAD DATA INFILE 'C:/MyWamp/mysql/script..
Affected 61296 rows
Running query:  CMySQLCursor: LOAD DATA INFILE 'C:/MyWamp/mysql/script..
Affected 61296 rows
Running query:  CMySQLCursor: (Nothing execu